In [1]:
import face_recognition
import cv2
import numpy as np
import os
import glob
import time
import imutils

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Get middle of screen coordinates
ret, frame = video_capture.read()
(h, w) = frame.shape[:2] #w:image-width and h:image-height
print(h,w)
mid_h = h/2
mid_w = w/2



480 640


In [2]:
# Draw Sudoku Puzzle expected location
left, top, right, bottom = (int(mid_w-150), int(mid_h+150), int(mid_w+150), int(mid_h-150))

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    # Apply grid to corner coordinates for user to place Sudoku Puzzle within
    # Added a padding because it was picking up the rectangle drawn on the frame
    cv2.rectangle(frame, (left-5, top+5), (right+5, bottom-5), (0, 0, 255), 2)
    
    # Apply filters to make grid detection easier
    cropped_frame = frame[bottom:top,left:right]
    gray_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY) 
    blur_frame = cv2.GaussianBlur(gray_frame, (5,5), 0)
    thresh_frame = cv2.adaptiveThreshold(blur_frame, 255, 1, 1, 11, 2)
    
    contours, hierarchy = cv2.findContours(thresh_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_areas = [cv2.contourArea(contour) for contour in contours]
    max_contour = None
    if len(contours) > 0:
        max_contour = [contours[contour_areas.index(max(contour_areas))]]
    
    # Check to see if the contour occupies > 80% of image area
    if max_contour != None and max(contour_areas) > 300*300*0.8:
        cv2.drawContours(cropped_frame, max_contour, 0 , (0, 255, 0), 3)
        
        mask = np.zeros((gray_frame.shape),np.uint8)
        # Fill in the pixels within the contour with white
        cv2.drawContours(mask,max_contour,0,255,-1)
        # Color over the pixels at the border of the contour with black
        cv2.drawContours(mask,max_contour,0,0,4)

        # Detect corner edges for perspective transformation
        corners = cv2.goodFeaturesToTrack(mask,4,0.2,100)
 
        # Conduct perspective transform
        try:
            M = cv2.getPerspectiveTransform(np.float32(corners),np.float32([[0,0],[300,0],[0,300],[300,300]]))
            dst = cv2.warpPerspective(cropped_frame,M,(300,300))
            
        except:
            dst = dst


        cv2.imshow('Video3', dst)
        time.sleep(2)
        
    # What the user sees
    cv2.imshow('Video2', frame)
    
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

cv2.imwrite('sudoku_thresh.jpg', dst)

True

In [3]:
sudoku_image = cv2.imread('expert_sudoku.png')
sudoku_image_raw = sudoku_image

cv2.imshow('sudoku_image_raw',sudoku_image_raw)
cv2.waitKey()

119

In [4]:
# Processing image for line detection

gray_frame = cv2.cvtColor(sudoku_image, cv2.COLOR_BGR2GRAY) 
blur_frame = cv2.GaussianBlur(gray_frame, (5,5), 0)
thresh_frame = cv2.adaptiveThreshold(blur_frame, 255, 1, 1, 11, 2)
edges = cv2.Canny(thresh_frame,90,150,apertureSize = 5)
cv2.imshow('canny.jpg',edges)
kernel = np.ones((3,3),np.uint8)
edges = cv2.dilate(edges,kernel,iterations = 1)
cv2.imshow('dilated.jpg',edges)
kernel = np.ones((3,3),np.uint8)
edges = cv2.erode(edges,kernel,iterations = 1)
cv2.imshow('eroded.jpg',edges)

lines = cv2.HoughLines(edges,1,np.pi/180,150)

if not lines.any():
    print('No lines were found')
    exit()

if filter:
    rho_threshold = 15
    theta_threshold = 0.1

    # how many lines are similar to a given one
    similar_lines = {i : [] for i in range(len(lines))}
    for i in range(len(lines)):
        for j in range(len(lines)):
            if i == j:
                continue

            rho_i,theta_i = lines[i][0]
            rho_j,theta_j = lines[j][0]
            if abs(rho_i - rho_j) < rho_threshold and abs(theta_i - theta_j) < theta_threshold:
                similar_lines[i].append(j)

    # ordering the indices of the lines by how many are similar to them
    indices = [i for i in range(len(lines))]
    indices.sort(key=lambda x : len(similar_lines[x]))

    # line flags is the base for the filtering
    line_flags = len(lines)*[True]
    for i in range(len(lines) - 1):
        if not line_flags[indices[i]]: # if we already disregarded the ith element in the ordered list 
            continue

        for j in range(i + 1, len(lines)): # we are only considering those elements that had less similar line
            if not line_flags[indices[j]]: # and only if we have not disregarded them already
                continue

            rho_i,theta_i = lines[indices[i]][0]
            rho_j,theta_j = lines[indices[j]][0]
            if abs(rho_i - rho_j) < rho_threshold and abs(theta_i - theta_j) < theta_threshold:
                line_flags[indices[j]] = False # if it is similar and have not been disregarded yet then drop it now

print('number of Hough lines:', len(lines))

filtered_lines = []

if filter:
    for i in range(len(lines)): # filtering
        if line_flags[i]:
            filtered_lines.append(lines[i])

    print('Number of filtered lines:', len(filtered_lines))
else:
    filtered_lines = lines

vertical_line_origins = []
horizontal_line_origins = []

for line in filtered_lines:
    rho,theta = line[0]
    if (theta > -np.pi/144 and theta < np.pi/144): 
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(sudoku_image,(x1,y1),(x2,y2),(255,255,255),2)
        horizontal_line_origins.append((0,x0))
        
    if (theta > np.pi/2-np.pi/144 and theta < np.pi/2+np.pi/144):
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(sudoku_image,(x1,y1),(x2,y2),(255,255,255),2)
        vertical_line_origins.append((y0,0))
        print(x0,y0)
        
cv2.imshow('sudoku_puzzle with lines',sudoku_image)
cv2.waitKey()

NameError: name 'sudoku_image' is not defined

In [ ]:
# Line check to make sure expected number of lines was detected and are spaced reasonably

vertical_line_origins.sort()
vertical_line_origins
for coords_id in range(len(vertical_line_origins)-1,0,-1):
    # Make sure vertical lines are not too close to each other, 50% buffer
    if vertical_line_origins[coords_id][0] - vertical_line_origins[coords_id-1][0] < 300/15:
        vertical_line_origins.pop(coords_id)

if len(vertical_line_origins) != 10:
    print("Some vertical lines not detected.")

horizontal_line_origins.sort()
horizontal_line_origins
for coords_id in range(len(horizontal_line_origins)-1,0,-1):
    # Make sure horizontal lines are not too close to each other, 50% buffer
    if horizontal_line_origins[coords_id][1] - horizontal_line_origins[coords_id-1][1] < 300/15:
        horizontal_line_origins.pop(coords_id)

if len(horizontal_line_origins) != 10:
    print("Some horizontal lines not detected.")
print(horizontal_line_origins)
print(vertical_line_origins)

In [ ]:
# Crop little boxes out for each cell

cropped_cells_coords = []
cropped_cells_images = []

for x_coords_id in range(0,len(vertical_line_origins)-1):
    for y_coords_id in range(0,len(horizontal_line_origins)-1):
        left = int(vertical_line_origins[x_coords_id][0])
        right = int(vertical_line_origins[x_coords_id+1][0])
        bottom = int(horizontal_line_origins[y_coords_id][1])
        top = int(horizontal_line_origins[y_coords_id+1][1])
                 
        # Pixel counts start from 1 for cropping
        if left == 0:
            left = 1
        if right == 0:
            right= 1
        if top == 0:
            top = 1
        if bottom == 0:
            bottom = 1
            
        cropped_cells_coords.append((bottom,top,left,right)) 
        cropped_cell_image = sudoku_image[int(bottom):int(top),int(left):int(right)]
        #cropped_cell_image = cv2.rotate(cropped_cell_image,cv2.ROTATE_180)
        #cropped_cell_image = cv2.flip(cropped_cell_image, 1)
        average_pixel = cropped_cell_image.mean(axis=0).mean(axis=0)
        average_pixel = [1.2*pixel_brightness for pixel_brightness in average_pixel]
        cv2.rectangle(cropped_cell_image,(0,0),(cropped_cell_image.shape[1],cropped_cell_image.shape[0]),average_pixel,6)
        rotated = imutils.rotate_bound(cropped_cell_image, 0)
        gray_frame = cv2.cvtColor(rotated,cv2.COLOR_BGR2GRAY)
        average_pixel = gray_frame.mean(axis=0).mean(axis=0)*0.9
        ret,thresh_frame = cv2.threshold(gray_frame,average_pixel,255,cv2.THRESH_BINARY)
        thresh_frame = cv2.bitwise_not(thresh_frame)
        
        #kernel = np.ones((2,2), np.uint8)
        #thresh_frame = cv2.erode(thresh_frame, kernel, iterations=1)
        #thresh_frame = cv2.dilate(thresh_frame, kernel, iterations=1)


        #thresh_frame = cv2.bitwise_not(thresh_frame)
        
        
        cropped_cells_images.append(thresh_frame)


for cropped_cell_image in cropped_cells_images[:10]:
    cv2.imshow('cell',cropped_cell_image)
    cv2.waitKey()

In [17]:
# Record list of predicted values for sudoku_input
# Wanted to find best orientation based on 'most confident angle', but model predictions are too confidently wrong
# Used tanh activation functions in  model instead of relu to decrease confidence of predictions, but decrease made predictions slightly more unreliable

import imutils
import tensorflow
from tensorflow import keras

# File path (number_predictor, number_predictor_soft = my own model)
filepath = 'C:\\Users\\jooer\\OneDrive\\Desktop\\MACHINE LEARNING\\computer_vision\\sudoku_solver\\number_predictor_soft.h5'

# Load the number recognition model
model = keras.models.load_model(filepath, compile = True)
filled_cells = 0
sudoku_input = []

# Access each cropped cell and predict the value in each cell, store in sudoku_input
for cropped_cell_image in cropped_cells_images:
    if np.sum(cropped_cell_image == (255)) > cropped_cell_image.shape[1]*cropped_cell_image.shape[0]*0.01:
        cropped_cell_image = cv2.resize(cropped_cell_image, (28,28), interpolation = cv2.INTER_AREA)        
        cropped_cell_image = cropped_cell_image[:, :, np.newaxis]

        #cv2.imshow("Cell with Number", cropped_cell_image)
        #cv2.waitKey(0)
        model_inputs = []
        model_inputs.append(cropped_cell_image)
        model_inputs = np.array(model_inputs)

        predictions = model.predict(model_inputs)
        classes = np.argmax(predictions, axis = 1)
        print(predictions)
        print(max(predictions[0]),classes)

        filled_cells+=1
        
    else:
        classes = [0]
        
    sudoku_input.append(classes[0])

[[2.8236941e-06 6.4879358e-01 1.3506572e-02 1.0418298e-04 8.0306432e-04
  2.0595471e-07 2.2897708e-09 3.3535755e-01 3.8254384e-06 1.4281655e-03]]
0.6487936 [1]
[[2.4905647e-01 1.0305686e-05 1.9980773e-02 2.1934426e-05 3.4398792e-04
  2.0757694e-02 6.5253198e-01 9.5200051e-07 5.7193134e-02 1.0273505e-04]]
0.652532 [6]
[[9.5240666e-06 2.4061077e-04 6.2317410e-03 2.9285314e-07 9.9174470e-01
  4.4672724e-08 1.2337981e-05 8.8583649e-05 1.5691251e-06 1.6707665e-03]]
0.9917447 [4]
[[8.8833326e-07 6.5072941e-06 3.8617521e-03 9.9598843e-01 3.8871562e-07
  3.9587914e-05 5.5149183e-09 7.3772862e-05 6.2244103e-06 2.2707318e-05]]
0.9959884 [3]
[[5.1440002e-05 8.8840046e-08 3.7358180e-03 2.6666204e-04 8.2601576e-07
  2.1535247e-04 4.5267243e-05 4.5120657e-07 9.9568057e-01 3.4133957e-06]]
0.9956806 [8]
[[7.0634211e-04 3.6166937e-06 9.6280808e-03 1.2636558e-02 8.8423248e-03
  4.8988391e-02 1.7891965e-06 3.8958728e-04 2.0616921e-03 9.1674167e-01]]
0.91674167 [9]
[[5.5217315e-09 9.5178548e-06 1.0309384e

In [18]:
# Create sudoku_grid with planned inputs for insertion

sudoku_grid = [[['row '+ str(row) + ' col ' + str(col)] for col in range(1,10)] for row in range(1,10)]

for row in range(0,9):
    print(sudoku_grid[row])
print(sudoku_input)

[['row 1 col 1'], ['row 1 col 2'], ['row 1 col 3'], ['row 1 col 4'], ['row 1 col 5'], ['row 1 col 6'], ['row 1 col 7'], ['row 1 col 8'], ['row 1 col 9']]
[['row 2 col 1'], ['row 2 col 2'], ['row 2 col 3'], ['row 2 col 4'], ['row 2 col 5'], ['row 2 col 6'], ['row 2 col 7'], ['row 2 col 8'], ['row 2 col 9']]
[['row 3 col 1'], ['row 3 col 2'], ['row 3 col 3'], ['row 3 col 4'], ['row 3 col 5'], ['row 3 col 6'], ['row 3 col 7'], ['row 3 col 8'], ['row 3 col 9']]
[['row 4 col 1'], ['row 4 col 2'], ['row 4 col 3'], ['row 4 col 4'], ['row 4 col 5'], ['row 4 col 6'], ['row 4 col 7'], ['row 4 col 8'], ['row 4 col 9']]
[['row 5 col 1'], ['row 5 col 2'], ['row 5 col 3'], ['row 5 col 4'], ['row 5 col 5'], ['row 5 col 6'], ['row 5 col 7'], ['row 5 col 8'], ['row 5 col 9']]
[['row 6 col 1'], ['row 6 col 2'], ['row 6 col 3'], ['row 6 col 4'], ['row 6 col 5'], ['row 6 col 6'], ['row 6 col 7'], ['row 6 col 8'], ['row 6 col 9']]
[['row 7 col 1'], ['row 7 col 2'], ['row 7 col 3'], ['row 7 col 4'], ['row 7

In [19]:
# Insert sudoku_input into sudoku_grid
for row in range(0,9):
    for col in range(0,9):
        sudoku_grid[row][col] = sudoku_input[(row) + (col)*9]
 

In [25]:
# Manual fixes for wrongly predicted values, types of errors vary by font/writing :']

sudoku_grid[2][7] = 1
sudoku_grid[8][3] = 1

sudoku_grid

[[0, 0, 0, 0, 0, 0, 6, 3, 4],
 [0, 0, 0, 0, 9, 1, 0, 8, 0],
 [0, 0, 0, 0, 0, 2, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 7],
 [0, 0, 8, 0, 0, 0, 4, 0, 0],
 [0, 0, 9, 7, 0, 5, 0, 0, 0],
 [0, 0, 0, 0, 6, 0, 0, 0, 0],
 [0, 4, 0, 0, 3, 0, 0, 0, 8],
 [6, 3, 0, 1, 0, 0, 0, 0, 0]]

In [36]:
# Implement basic rule (row, col, grid) checker for all empty cells (cell value = 0)
import copy

def basic_check(sudoku_grid_input):
    detault_set = {1,2,3,4,5,6,7,8,9}
    
    sudoku_grid_input = copy.deepcopy(sudoku_grid_input)
    # Reset grid - should probably refactor code such that the initial expected sudoku_grid cell data is already in sets
    for row_id in range(0,len(sudoku_grid_input)):
            for col_id in range(0,len(sudoku_grid_input[0])):
                if type(sudoku_grid_input[row_id][col_id]) == set:
                    sudoku_grid_input[row_id][col_id] = 0
                    
                
    updated_sudoku_grid = copy.deepcopy(sudoku_grid_input)

    for row_id in range(0,len(sudoku_grid_input)):
        for col_id in range(0,len(sudoku_grid_input[0])):
            
            if sudoku_grid_input[row_id][col_id] == 0:
                row_check_set = set(sudoku_grid_input[row_id])
                column_check_set = set()
                other_row_ids = {0,1,2,3,4,5,6,7,8} - {row_id}
                for other_row_id in other_row_ids:
                    column_check_set = column_check_set | {sudoku_grid_input[other_row_id][col_id]}

                block_check_set = set()
                for block_rows in range(int((row_id)/3)*3,int((row_id)/3)*3+2+1):
                    for block_cols in range(int((col_id)/3)*3,int((col_id)/3)*3+2+1):
                        block_check_set = block_check_set | {sudoku_grid_input[block_rows][block_cols]}

                updated_sudoku_grid[row_id][col_id] = detault_set - row_check_set - column_check_set - block_check_set
    
    for row_id in range(0,len(updated_sudoku_grid)):
        for col_id in range(0,len(updated_sudoku_grid[0])):
            if type(updated_sudoku_grid[row_id][col_id]) == set:
                if len(updated_sudoku_grid[row_id][col_id]) == 1:
                    for value in updated_sudoku_grid[row_id][col_id]:
                        updated_sudoku_grid[row_id][col_id] = value
            
    return(updated_sudoku_grid)

In [43]:
# Apply basic rules repeatedly until no more cells can be filled

grid_changed = True
while grid_changed == True:
    updated_sudoku_grid = basic_check(sudoku_grid)
    if updated_sudoku_grid == sudoku_grid:
        grid_changed = False  
    sudoku_grid = copy.deepcopy(updated_sudoku_grid)


sudoku_grid

[[{2, 5, 7, 8, 9},
  {1, 2, 5, 7, 8, 9},
  {1, 2, 5, 7},
  {5, 8},
  {5, 7, 8},
  {7, 8},
  6,
  3,
  4],
 [{2, 3, 4, 5, 7},
  {2, 5, 6, 7},
  {2, 3, 4, 5, 6, 7},
  {3, 4, 5, 6},
  9,
  1,
  {2, 5, 7},
  8,
  {2, 5}],
 [{3, 4, 5, 7, 8, 9},
  {5, 6, 7, 8, 9},
  {3, 4, 5, 6, 7},
  {3, 4, 5, 6, 8},
  {4, 5, 7, 8},
  2,
  {5, 7, 9},
  1,
  {5, 9}],
 [1,
  {2, 5, 6},
  {2, 3, 4, 5, 6},
  {2, 3, 4, 6, 8, 9},
  {2, 4, 8},
  {3, 4, 6, 8, 9},
  {2, 3, 5, 8, 9},
  {2, 5, 6, 9},
  7],
 [{2, 3, 5, 7},
  {2, 5, 6, 7},
  8,
  {2, 3, 6, 9},
  {1, 2},
  {3, 6, 9},
  4,
  {2, 5, 6, 9},
  {1, 2, 3, 5, 6, 9}],
 [{2, 3, 4},
  {2, 6},
  9,
  7,
  {1, 2, 4, 8},
  5,
  {1, 2, 3, 8},
  {2, 6},
  {1, 2, 3, 6}],
 [{2, 5, 7, 8, 9},
  {1, 2, 5, 7, 8, 9},
  {1, 2, 5, 7},
  {2, 4, 5, 8, 9},
  6,
  {4, 7, 8, 9},
  {1, 2, 3, 5, 7, 9},
  {2, 4, 5, 7, 9},
  {1, 2, 3, 5, 9}],
 [{2, 5, 7, 9},
  4,
  {1, 2, 5, 7},
  {2, 5, 9},
  3,
  {7, 9},
  {1, 2, 5, 7, 9},
  {2, 5, 6, 7, 9},
  8],
 [6,
  3,
  {2, 5, 7},
  1,
  {2, 4, 

In [31]:
# Advanced logic 1: If a grid has 3 numbers filled in row, the numbers in adjacent grids are confirmed as options for that adjacent grid



True

In [ ]:
# Advanced Logic 2: In a row, column or grid, if there are sets of possibilites that are identical, and the number of sets = the number of identical items in the sets, the identical items are removed from all other sets in that row, column or grid

In [57]:
# Advanced Logic 3: In a row, column or grid, if there exist a value in a set that is not existent in other sets, that cell with the set containing the unique value will take that unique value
# Everything will be dealt with as sets from this function on, each set is the space of values a cell can take

import copy

def logic3_check(sudoku_grid_input):
    updated_sudoku_grid = copy.deepcopy(sudoku_grid_input)
    
    # Convert every cell to a set of possible values
    for row_id in range(0,len(sudoku_grid_input)):
        for col_id in range(0,len(sudoku_grid_input[0])):
            if type(updated_sudoku_grid[row_id][col_id]) != set:
                updated_sudoku_grid[row_id][col_id] = {updated_sudoku_grid[row_id][col_id]}
                        
    for row_id in range(0,len(updated_sudoku_grid)):
        for col_id in range(0,len(updated_sudoku_grid[0])):
            
            # If cell has more than one possibility, see if we can remove more possibilities with Logic 3
            if len(updated_sudoku_grid[row_id][col_id]) > 1:
                row_check_set = set()
                other_col_ids = {0,1,2,3,4,5,6,7,8} - {col_id}
                for other_col_id in other_col_ids:
                    row_check_set = row_check_set | updated_sudoku_grid[row_id][other_col_id]
                
                if len(updated_sudoku_grid[row_id][col_id] - row_check_set) == 1:
                    updated_sudoku_grid[row_id][col_id] = updated_sudoku_grid[row_id][col_id] - row_check_set
                    print('Row Check Triggered for: ' + str((row_id,col_id)))
                   
                column_check_set = set()
                other_row_ids = {0,1,2,3,4,5,6,7,8} - {row_id}
                for other_row_id in other_row_ids:
                    column_check_set = column_check_set | updated_sudoku_grid[other_row_id][col_id]
                    
                if len(updated_sudoku_grid[row_id][col_id] - column_check_set) == 1:
                    updated_sudoku_grid[row_id][col_id] = updated_sudoku_grid[row_id][col_id] - column_check_set
                    print('Col Check Triggered for: ' + str((row_id,col_id)))

                block_check_set = set()
                for block_rows in range(int((row_id)/3)*3,int((row_id)/3)*3+2+1):
                    for block_cols in range(int((col_id)/3)*3,int((col_id)/3)*3+2+1):
                        if (block_cols,block_rows) != (row_id,col_id):
                            block_check_set = block_check_set | updated_sudoku_grid[block_rows][block_cols]
                            print(str(block_check_set) + str((block_rows,block_cols)))

                if len(updated_sudoku_grid[row_id][col_id] - block_check_set) == 1:
                    print(updated_sudoku_grid[row_id][col_id])
                    print(block_check_set)
                    updated_sudoku_grid[row_id][col_id] = updated_sudoku_grid[row_id][col_id] - block_check_set
                    print('Block Check Triggered for: ' + str((row_id,col_id)))
            
    return(updated_sudoku_grid)

sudoku_grid = logic3_check(sudoku_grid)

{1, 2, 5, 7, 8, 9}(0, 1)
{1, 2, 5, 7, 8, 9}(0, 2)
{1, 2, 3, 4, 5, 7, 8, 9}(1, 0)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(1, 1)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(1, 2)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 0)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 1)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 2)
{2, 5, 7, 8, 9}(0, 0)
{1, 2, 5, 7, 8, 9}(0, 1)
{1, 2, 5, 7, 8, 9}(0, 2)
{1, 2, 5, 6, 7, 8, 9}(1, 1)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(1, 2)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 0)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 1)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 2)
{2, 5, 7, 8, 9}(0, 0)
{1, 2, 5, 7, 8, 9}(0, 1)
{1, 2, 5, 7, 8, 9}(0, 2)
{1, 2, 3, 4, 5, 7, 8, 9}(1, 0)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(1, 1)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(1, 2)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 1)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 2)
{8, 5}(0, 3)
{5, 7, 8}(0, 4)
{5, 7, 8}(0, 5)
{3, 4, 5, 6, 7, 8}(1, 3)
{3, 4, 5, 6, 7, 8, 9}(1, 4)
{1, 3, 4, 5, 6, 7, 8, 9}(1, 5)
{1, 3, 4, 5, 6, 7, 8, 9}(2, 3)
{1, 3, 4, 5, 6, 7, 8, 9}(2, 4)
{1, 2, 3, 4, 5, 6, 7, 8, 9}(2, 5)
{8, 5}(0, 3)
{5, 7, 8}(0, 4)
{5, 7, 8}(0, 5)
{3, 

In [58]:
grid_changed = True
while grid_changed == True:
    updated_sudoku_grid = basic_check(sudoku_grid)
    if updated_sudoku_grid == sudoku_grid:
        grid_changed = False  
    sudoku_grid = copy.deepcopy(updated_sudoku_grid)


sudoku_grid

[[{1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9}],
 [{1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9}],
 [{1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9}],
 [{1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1, 2, 3, 4, 5, 6, 7, 8, 9},
  {1,

In [42]:
{1,2,3} - {3,4,5}

{1, 2}

In [52]:
True and False

False